# Monetary Policy and Corporate Investment Decisions (2020–2025)

**Project Overview**  
This notebook examines how Federal Reserve monetary policy changes influence corporate capital expenditure (capex) decisions, using the Magnificent Seven tech companies as a case study. It builds on Projects 1 & 2 by incorporating policy shocks and macroeconomic indicators as predictors of firm-level investment behaviour.

**Data Sources**  
- FRED: Federal Funds Rate, Real GDP  
- Yahoo Finance: Stock prices & fundamentals for AAPL, MSFT, GOOGL, AMZN, NVDA, META, TSLA

**Methodology**  
1. Construct quarterly panel dataset with capex growth, policy changes, GDP growth, and stock returns  
2. Panel regression with ticker fixed effects: Capex_Growth ~ Delta_FedFunds + GDP_Growth + Return  
3. Visualise historical capex trends and correlations with policy shifts  
4. Simulate forward-looking scenarios (e.g., 'Tightening cycle predicts capex cuts')  

In [1]:
import sys
from pathlib import Path
import pandas as pd
import plotly.express as px

repo_root = Path().resolve().parent
sys.path.insert(0, str(repo_root))

In [2]:
from src.corporate_decisions.data_loader import DataLoader
from src.corporate_decisions.data_analysis import DataAnalysis
from src.corporate_decisions.simulate_events import EventSimulator

## 1. Data Loading & Panel Construction

We begin by fetching combined financial data (stock prices, fundamentals, and macro indicators) and constructing a quarterly panel dataset suitable for regression analysis.

In [3]:
data_loader = DataLoader()
data_loader.load_data()

In [4]:
# Initialise analysis object and build quarterly panel
data_dir = repo_root / "data" / "corporate_decisions"
analysis = DataAnalysis(data_dir)

# Build panel (creates mag7_panel_quarterly.csv)
panel = analysis.build_panel()

## 2. Exploratory Data Analysis

### 2.1 Historical Capex Growth Trends

In [5]:
analysis.visualise_panel(panel)

### 2.2 Correlation with Monetary Policy Changes

Examine how capex growth correlates with Fed rate changes across the Magnificent Seven.

In [6]:
import plotly.graph_objects as go

# Aggregate average capex growth and rate changes by quarter
quarterly_agg = panel.reset_index().groupby('Date').agg({
    'Capex_Growth': 'mean',
    'Delta_FedFunds': 'first',
    'GDP_Growth': 'first'
}).dropna()

# Dual-axis plot: capex growth vs rate changes
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=quarterly_agg.index,
    y=quarterly_agg['Capex_Growth'] * 100,
    name='Avg Capex Growth (%)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=quarterly_agg.index,
    y=quarterly_agg['Delta_FedFunds'],
    name='Δ Fed Funds Rate (pp)',
    yaxis='y2',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='Corporate Capex Growth vs Monetary Policy Changes',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Avg Capex Growth (%)', side='left'),
    yaxis2=dict(title='Δ Fed Funds (pp)', side='right', overlaying='y'),
    hovermode='x unified',
    height=500
)

fig.show()

print(f"Correlation (Capex Growth, Δ Fed Funds): {quarterly_agg['Capex_Growth'].corr(quarterly_agg['Delta_FedFunds']):.3f}")

Correlation (Capex Growth, Δ Fed Funds): -0.436


**Interpretation**: A negative correlation suggests that rate hikes (positive Δ Fed Funds) tend to coincide with slower capex growth, consistent with higher financing costs reducing corporate investment appetite.

## 3. Panel Regression Analysis

### Model Specification
$$
\text{Capex Growth}_{it} = \beta_0 + \beta_1 \Delta \text{FedFunds}_t + \beta_2 \text{GDP Growth}_t + \beta_3 \text{Return}_{it} + \alpha_i + \epsilon_{it}
$$

where $\alpha_i$ are ticker fixed effects (capturing firm-specific investment patterns) and standard errors are clustered by ticker.

## 4. Forward-Looking Scenario Simulations

### Scenario Design
We simulate quarterly capex growth under three hypothetical 2026 policy paths:

1. **Tightening Cycle**: Fed hikes 100 bp over 4 quarters (25 bp/quarter)  
2. **Easing Cycle**: Fed cuts 100 bp over 4 quarters (-25 bp/quarter)  
3. **Status Quo**: No rate changes (hold at current level)

The simulation uses the fitted panel regression coefficients and propagates policy changes forward, assuming GDP growth remains at the recent average and stock returns follow historical volatility patterns.

In [7]:
# Initialize event simulator
data_directory = Path("data/corporate_decisions")
simulator = EventSimulator(data_directory=data_directory)

                            OLS Regression Results                            
Dep. Variable:           Capex_Growth   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                     6.482
Date:                Sat, 14 Feb 2026   Prob (F-statistic):             0.0260
Time:                        00:16:55   Log-Likelihood:                -8.3821
No. Observations:                  80   AIC:                             36.76
Df Residuals:                      70   BIC:                             60.58
Df Model:                           9                                         
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.0514      0

/Users/rayanjoshi/repos/dreamy_veil/src/corporate_decisions/simulate_events.py:37: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/Users/rayanjoshi/repos/dreamy_veil/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 9, but rank is 3



### Scenario 1: Tightening Cycle (4 × 25 bp hikes)

In [8]:
tightening_sim = simulator.simulate_event(
    event_type='tightening',
    quarters=4,
    rate_change_per_quarter=0.25,
    gdp_growth_assumption=0.005  # 0.5% quarterly GDP growth
)

print("\n=== Tightening Cycle Simulation ===")
print(tightening_sim[['Ticker', 'Quarter', 'Predicted_Capex_Growth', 'Cumulative_Capex_Change']].head(12))


=== Tightening Cycle Simulation ===
   Ticker  Quarter  Predicted_Capex_Growth  Cumulative_Capex_Change
0    AAPL        1               -0.008103                -0.008103
1    MSFT        1                0.006908                 0.006908
2   GOOGL        1                0.013361                 0.013361
3    AMZN        1                0.006579                 0.006579
4    NVDA        1                0.009147                 0.009147
5    META        1                0.004583                 0.004583
6    TSLA        1                0.014765                 0.014765
7    AAPL        2                0.000860                -0.007250
8    MSFT        2                0.011545                 0.018532
9   GOOGL        2                0.008557                 0.022032
10   AMZN        2                0.007780                 0.014410
11   NVDA        2                0.003794                 0.012976


In [9]:
simulator.plot_simulation(tightening_sim, title='Scenario 1: Tightening Cycle (100 bp hikes)')

### Scenario 2: Easing Cycle (4 × 25 bp cuts)

In [10]:
easing_sim = simulator.simulate_event(
    event_type='easing',
    quarters=4,
    rate_change_per_quarter=-0.25,
    gdp_growth_assumption=0.005
)

print("\n=== Easing Cycle Simulation ===")
print(easing_sim[['Ticker', 'Quarter', 'Predicted_Capex_Growth', 'Cumulative_Capex_Change']].head(12))


=== Easing Cycle Simulation ===
   Ticker  Quarter  Predicted_Capex_Growth  Cumulative_Capex_Change
0    AAPL        1                0.054526                 0.054526
1    MSFT        1                0.069537                 0.069537
2   GOOGL        1                0.075990                 0.075990
3    AMZN        1                0.069208                 0.069208
4    NVDA        1                0.071776                 0.071776
5    META        1                0.067211                 0.067211
6    TSLA        1                0.077393                 0.077393
7    AAPL        2                0.063489                 0.121477
8    MSFT        2                0.074174                 0.148868
9   GOOGL        2                0.071186                 0.152585
10   AMZN        2                0.070409                 0.144490
11   NVDA        2                0.066423                 0.142966


In [11]:
simulator.plot_simulation(easing_sim, title='Scenario 2: Easing Cycle (100 bp cuts)')

### Scenario 3: Status Quo (No rate changes)

In [12]:
status_quo_sim = simulator.simulate_event(
    event_type='status_quo',
    quarters=4,
    rate_change_per_quarter=0.0,
    gdp_growth_assumption=0.005
)

print("\n=== Status Quo Simulation ===")
print(status_quo_sim[['Ticker', 'Quarter', 'Predicted_Capex_Growth', 'Cumulative_Capex_Change']].head(12))


=== Status Quo Simulation ===
   Ticker  Quarter  Predicted_Capex_Growth  Cumulative_Capex_Change
0    AAPL        1                0.023212                 0.023212
1    MSFT        1                0.038222                 0.038222
2   GOOGL        1                0.044675                 0.044675
3    AMZN        1                0.037894                 0.037894
4    NVDA        1                0.040462                 0.040462
5    META        1                0.035897                 0.035897
6    TSLA        1                0.046079                 0.046079
7    AAPL        2                0.032175                 0.056133
8    MSFT        2                0.042860                 0.082720
9   GOOGL        2                0.039871                 0.086328
10   AMZN        2                0.039095                 0.078470
11   NVDA        2                0.035108                 0.076990


In [13]:
simulator.plot_simulation(status_quo_sim, title='Scenario 3: Status Quo (No rate changes)')

### Comparative Scenario Summary

In [14]:
# Compare cumulative capex changes across scenarios
summary_data = []

for scenario_name, sim_df in [
    ('Tightening', tightening_sim),
    ('Easing', easing_sim),
    ('Status Quo', status_quo_sim)
]:
    final_quarter = sim_df['Quarter'].max()
    avg_final_change = sim_df[sim_df['Quarter'] == final_quarter]['Cumulative_Capex_Change'].mean()
    summary_data.append({
        'Scenario': scenario_name,
        'Avg Cumulative Capex Change (%)': round(avg_final_change * 100, 2)
    })

summary_df = pd.DataFrame(summary_data)
print("\n=== Scenario Comparison: Avg Capex Impact After 4 Quarters ===")
print(summary_df.to_string(index=False))

# Bar chart comparison

fig = px.bar(
    summary_df,
    x='Scenario',
    y='Avg Cumulative Capex Change (%)',
    title='Capex Impact by Policy Scenario (4-Quarter Horizon)',
    color='Avg Cumulative Capex Change (%)',
    color_continuous_scale='RdYlGn'
)
fig.update_layout(height=400)
fig.show()


=== Scenario Comparison: Avg Capex Impact After 4 Quarters ===
  Scenario  Avg Cumulative Capex Change (%)
Tightening                             2.60
    Easing                            30.63
Status Quo                            15.98


## 5. Sector-Level and Firm-Level Heterogeneity

### Which firms are most sensitive to policy changes?

In [15]:
# Calculate firm-specific sensitivity to rate changes
firm_sensitivity = []

for ticker in panel.reset_index()['Ticker'].unique():
    firm_data = panel.reset_index()[panel.reset_index()['Ticker'] == ticker].copy()
    
    # Simple correlation between capex growth and rate changes
    corr = firm_data[['Capex_Growth', 'Delta_FedFunds']].corr().iloc[0, 1]
    
    # Average capex growth volatility
    volatility = firm_data['Capex_Growth'].std()
    
    firm_sensitivity.append({
        'Ticker': ticker,
        'Policy Sensitivity (ρ)': round(corr, 3),
        'Capex Volatility (σ)': round(volatility, 3)
    })

sensitivity_df = pd.DataFrame(firm_sensitivity).sort_values('Policy Sensitivity (ρ)')

print("\n=== Firm-Level Policy Sensitivity ===")
print(sensitivity_df.to_string(index=False))
print("\nInterpretation: Negative ρ means rate hikes → lower capex growth")


=== Firm-Level Policy Sensitivity ===
Ticker  Policy Sensitivity (ρ)  Capex Volatility (σ)
 GOOGL                  -0.765                 0.267
  AMZN                  -0.747                 0.238
  META                  -0.661                 0.272
  MSFT                  -0.173                 0.203
  NVDA                  -0.144                 0.642
  TSLA                  -0.090                 0.131
  AAPL                  -0.049                 0.111

Interpretation: Negative ρ means rate hikes → lower capex growth


In [16]:
# Visualise firm sensitivity
fig = px.scatter(
    sensitivity_df,
    x='Policy Sensitivity (ρ)',
    y='Capex Volatility (σ)',
    text='Ticker',
    title='Firm-Level Policy Sensitivity vs Capex Volatility',
    labels={
        'Policy Sensitivity (ρ)': 'Policy Sensitivity (Corr: Capex Growth, Δ Fed Funds)',
        'Capex Volatility (σ)': 'Capex Growth Std Dev'
    },
    height=500
)

fig.update_traces(textposition='top center', marker=dict(size=12))
fig.add_vline(x=0, line_dash='dash', line_color='gray')
fig.show()

**Findings**:  
- Firms in the **bottom-left quadrant** (negative ρ, low σ) are consistently sensitive to rate hikes but with stable capex patterns  
- Firms in the **top-right quadrant** (positive/neutral ρ, high σ) have volatile capex cycles less directly tied to Fed policy  
- Firms like TSLA and NVDA (high growth, capital-intensive) may show higher sensitivity due to greater reliance on external financing

## 6. Key Takeaways & Policy Implications

### Main Findings

1. **Negative Policy Transmission**: Rate hikes → Lower stock returns & higher financing costs → Reduced capex growth  

2. **Equity Market Channel**: Stock market reactions partially mediate policy impacts on investment  
   - Strong positive correlation between quarterly equity returns and capex growth

3. **Firm Heterogeneity**: Tech firms show varying sensitivity to monetary policy  
   - Capital-intensive firms (NVDA, TSLA) more sensitive to financing cost shocks  
   - Cash-rich firms (AAPL, MSFT) can partially buffer against policy headwinds

4. **Scenario Projections**: 
   - Tightening cycle (100 bp hikes) → Avg capex to decline. 
   - Easing cycle (100 bp cuts) → Avg capex to increase.


### Limitations & Extensions

**Current Limitations**:  
- Small sample (N=7 firms, tech-heavy)  
- Linear models assume constant coefficients (policy effects may be non-linear or time-varying)  
- No modelling of expectations or forward guidance  
- Capex data is annual/quarterly → limited granularity

**Potential Extensions**:  
1. Expand to broader S&P 500 sample using WRDS Compustat (500 firms, multiple sectors)  
2. Incorporate sector-specific interest rate sensitivities (e.g., utilities vs tech)  
3. Add credit spread and corporate bond yield data (high-yield spreads as additional channel)  
4. Use event study methodology around FOMC announcements for cleaner identification  